In [1]:
#hi, welcome to my notebook for scraping rotten tomatoes 
#install all the necessary packages 
!pip3 install -U selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
!pip3 install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import json
import numpy as np

In [14]:
#install chromedriver 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [13]:
#open up the rotten tomatoes home page 
#that we will be pulling movie urls and tomatometer ratings from
driver.get('https://www.rottentomatoes.com/browse/movies_at_home/sort:popular?page=3')

MaxRetryError: HTTPConnectionPool(host='localhost', port=63546): Max retries exceeded with url: /session/8b54da4e4dedf4e74ede32a02a20b815/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f850a0fa220>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [4]:
#on inspection of the website HTML, line 93 
#contains the urls and tomatometer ratings 
#for 50 movies displayed on the homepage
html_line93 = driver.find_elements(By.XPATH, '//script[@type="application/ld+json"]')
#get the HTML 
json_content = html_line93[0].get_attribute('innerHTML')
data = json.loads(json_content)
#to inspect the data in its raw form: print(data)

In [5]:
#create a dataframe from the data
df = pd.DataFrame(data)
df

,@context,@type,url,name,description,itemListElement
@type,http://schema.org,ItemList,https://www.rottentomatoes.com/browse/movies_a...,Best Movies to Stream at Home <span>(2024)</span>,Discovery - Check out Best Movies to Stream at...,ItemList
itemListElement,http://schema.org,ItemList,https://www.rottentomatoes.com/browse/movies_a...,Best Movies to Stream at Home <span>(2024)</span>,Discovery - Check out Best Movies to Stream at...,"[{'@type': 'Movie', 'position': 1, 'name': 'Sa..."
name,http://schema.org,ItemList,https://www.rottentomatoes.com/browse/movies_a...,Best Movies to Stream at Home <span>(2024)</span>,Discovery - Check out Best Movies to Stream at...,Movies


In [6]:
#you will notice that the actual data we want to access is nested
#within the dataframe

#access the data we want and create a new dataframe 
# we will call this dataframe 'nested'
nested_data = df.loc['itemListElement']['itemListElement']
nested = pd.DataFrame(nested_data)
nested

,@type,position,name,aggregateRating,dateCreated,image,url,video
0,Movie,1,Saltburn,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/5IYYj4UmhM5oQyJ0...,https://www.rottentomatoes.com/m/saltburn,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
1,Movie,2,Society of the Snow,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/Fu6AXk1vSqv9NdFS...,https://www.rottentomatoes.com/m/society_of_th...,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
2,Movie,3,The Holdovers,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/Gfe3Lf75Qo5Pb0fe...,https://www.rottentomatoes.com/m/the_holdovers,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
3,Movie,4,Good Grief,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/dkaR8dOy_hboN3sS...,https://www.rottentomatoes.com/m/good_grief_2023,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
4,Movie,5,Rebel Moon: Part One - A Child of Fire,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/2bpFxoybigCD78l_...,https://www.rottentomatoes.com/m/rebel_moon_pa...,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
5,Movie,6,Leave the World Behind,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/gjBl91cQPoUL3N1P...,https://www.rottentomatoes.com/m/leave_the_wor...,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
6,Movie,7,Killers of the Flower Moon,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/GGQKtXozu58SHWKH...,https://www.rottentomatoes.com/m/killers_of_th...,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
7,Movie,8,Eileen,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/8pEe52HVvSsoAtke...,https://www.rottentomatoes.com/m/eileen,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
8,Movie,9,Foe,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/304MqUZNifMG8ip4...,https://www.rottentomatoes.com/m/foe_2023,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
9,Movie,10,Dream Scenario,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/IIgJhkc98BZTFlSD...,https://www.rottentomatoes.com/m/dream_scenario,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."


In [8]:
#some rows contain NaN values instead of a dictionary. drop these rows from both dataframes
nested = nested.drop([22, 48])

In [ ]:
#note: rotten tomatoes occasionally changes the data on the hompage
#so you need to manually inspect which of the 50 movies have NaN and ensure those are dropped in the cell above.

In [10]:
#let us first access the box office earnings of these movies
#upon inspection, 18 of the movies do not contain info on box office statistics
#sadly, we have to drop them from our nested dataframe
nested = nested.drop([1, 3, 4, 5, 10, 13, 20, 23, 27, 28, 30, 31, 34, 37, 41, 42, 44,])
nested.reset_index(drop=True, inplace=True)
nested.head()

,@type,position,name,aggregateRating,dateCreated,image,url,video
0,Movie,1,Saltburn,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/5IYYj4UmhM5oQyJ0...,https://www.rottentomatoes.com/m/saltburn,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
1,Movie,3,The Holdovers,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/Gfe3Lf75Qo5Pb0fe...,https://www.rottentomatoes.com/m/the_holdovers,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
2,Movie,7,Killers of the Flower Moon,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/GGQKtXozu58SHWKH...,https://www.rottentomatoes.com/m/killers_of_th...,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
3,Movie,8,Eileen,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/8pEe52HVvSsoAtke...,https://www.rottentomatoes.com/m/eileen,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."
4,Movie,9,Foe,"{'@type': 'AggregateRating', 'bestRating': '10...",2023,https://resizing.flixster.com/304MqUZNifMG8ip4...,https://www.rottentomatoes.com/m/foe_2023,"{'@type': 'VideoObject', 'thumbnailUrl': 'http..."


In [12]:
#now we are left with 30 movies in the nested dataframe

#this is a for loop that accesses each movie's url from the dataframe
#it then asks chromedriver to access that url
#we inspected the website html to find the XPATH for box office
#this XPATH was then used to access the box office earnings
#and this value was accessed and then appended to a list
boxofficelist = []

for i in range(30):
    url = nested.iloc[i]['url']
    driver.get(url)
    boxoffice_element = driver.find_elements(By.XPATH,"//b[text()='Box Office (Gross USA):']/following-sibling::span")
    boxoffice_value = boxoffice_element[0].get_attribute('innerHTML')
    boxoffice_value
    boxofficelist.append(boxoffice_value)

MaxRetryError: HTTPConnectionPool(host='localhost', port=63546): Max retries exceeded with url: /session/8b54da4e4dedf4e74ede32a02a20b815/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f850a127310>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
print(boxofficelist)

In [ ]:
finalratingslist = []

#note: the tomatometer ratings data we want to access
#is contained within an 'aggregateRating' dictonary for each movie. 
#we need to pull the rating out of this dictionary

#this is a for loop that goes through each row (each dictionary)
#it isolates the dictionary and turns it into a list 'dic_list'
#it then takes the fourth value from dic_list
#and adds it to the final ratings list
#(the fourth value in the dictionary is the tomatometer rating)

for i in range(30):
    isolated_dic = nested.iloc[i]['aggregateRating']  # Accessing the dictionary in the current row
    dic_list = list(isolated_dic.values())
    finalratingslist.append(dic_list[4])

#print the finalratingslist after the loop finishes
print(finalratingslist)

In [ ]:
#hooray! we now have the box office earnings and the Tomatometer ratings of 30 movies.

#These are the box office (US gross) in millions - which we obtained when we ran this code
#['11.4', '18.5', '67.3', '1.4', '4.9', '0.043', '163.8', '326.1', '40.8', '9.9', '61.2', '183.6', '92.3', '636.2',
#'0.006', '7.3', '20.2', '47.6', '0.5447', '137.3', '50.2', '44.2', '1.5', '31.8', '28.1', '76.7', '174.5', '100.5',
#'20.3', '2.3']

#These are the Tomatometer scores
#['71', '96', '93', '85', '91', '23', '64', '93', '66', '96', '59', '57', '76', '88', '72', '62', '83', '94', '98', 
#'32', '70', '65', '98', '83', '75', '94', '70', '60', '99', '79']

In [ ]:
driver.quit()